<a href="https://colab.research.google.com/github/Moostafaaa/Sentence-transformer-and-text-clustering/blob/main/text_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans

def make_subgroups_with_clustering(dataset, num_clusters):
    # Perform clustering
    features = dataset
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(features)

    # Add cluster labels to the dataset
    dataset['Cluster'] = kmeans.labels_

    # Group by cluster label
    subgroups = dataset #.groupby('Cluster')

    return subgroups



In [ ]:
df1 = pd.read_csv("/content/sciq_short.csv")
df2 = pd.read_csv("/content/train.csv")
df = pd.concat([df2 , df1])

In [ ]:
df.drop(["Unnamed: 0.1","Unnamed: 0"],axis=1,inplace=True)

In [ ]:
df.drop(["context","len"],axis=1,inplace=True)

In [ ]:
df.shape

(11975, 7)

In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=68aabc0c4dfdc99c201085ef36af60a23f087e8b85a460b4a32c2f4d1000448a
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
SIM_MODEL = 'sentence-transformers/all-MiniLM-L6-v2'
DEVICE = 0
MAX_LENGTH = 384
BATCH_SIZE = 16
trn = df
model = SentenceTransformer(SIM_MODEL, device='cuda')
model.max_seq_length = MAX_LENGTH
model = model.half()
## Combine all answers
trn['answer_all'] = trn.apply(lambda x: " ".join([x['A'], x['B'], x['C'], x['D'], x['E']]), axis=1)
## Search using the prompt and answers to guide the search
trn['prompt_answer_stem'] = trn['prompt'] + " " + trn['answer_all']
question_embeddings = model.encode(trn.prompt_answer_stem.values, batch_size=BATCH_SIZE, device=DEVICE,
        show_progress_bar=True, convert_to_tensor=True, normalize_embeddings=True)
question_embeddings = question_embeddings.detach().cpu().numpy()


Batches:   0%|          | 0/749 [00:00<?, ?it/s]

In [ ]:
question_embeddings.shape

(11975, 384)

In [ ]:
labels = cluster_numpy_array(question_embeddings, 30)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
set(labels)

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29}

In [ ]:
target = set(labels[0:200])

In [ ]:
df['labels'] = labels

In [ ]:
def on(number):
  if number in target:
    return 1
  else:
    return 0

In [ ]:
df['on'] = df['labels'].apply(on)

In [ ]:
df[df['on'] == 1].tail(10)

,prompt,A,B,C,D,E,answer,answer_all,prompt_answer_stem,labels,on
11763,How often do partial lunar eclipses occur?,6 times a year,2 times a year,once a year,6 times a year,twice a year,E,6 times a year 2 times a year once a year 6 ti...,How often do partial lunar eclipses occur? 6 t...,8,1
11764,What are the trillions of bacteria living with...,intastinal flora,probiotic,microflora,intastinal flora,bacteria flora,E,intastinal flora probiotic microflora intastin...,What are the trillions of bacteria living with...,6,1
11765,What is the process by which rocks and other s...,pulling,freezing,picking,pulling,plucking,E,pulling freezing picking pulling plucking,What is the process by which rocks and other s...,13,1
11767,The jejunum is about 0.9 meters (3 feet) long ...,black at death,time.the at death,weeks at death,black at death,empty at death,E,black at death time.the at death weeks at deat...,The jejunum is about 0.9 meters (3 feet) long ...,4,1
11768,Which construction material previously used in...,Silica,Alkylphenols,Paints contains lead,Silica,asbestos,E,Silica Alkylphenols Paints contains lead Silic...,Which construction material previously used in...,18,1
11769,What is the number waves that pass a fixed poi...,combination frequency,heating frequency,wave tendency,combination frequency,wave frequency,E,combination frequency heating frequency wave t...,What is the number waves that pass a fixed poi...,3,1
11770,"The outer ear, or ear canal, carries sound to ...",lungs,aorta,ear lobe,lungs,eardrum,E,lungs aorta ear lobe lungs eardrum,"The outer ear, or ear canal, carries sound to ...",1,1
11771,What sport involves people quickly finding des...,patterning,sticking,mapping,patterning,orienteering,E,patterning sticking mapping patterning oriente...,What sport involves people quickly finding des...,2,1
11772,Almost all earthquakes occur at which place?,continental shelf,mountains,land boundaries,continental shelf,plate boundaries,E,continental shelf mountains land boundaries co...,Almost all earthquakes occur at which place? c...,13,1
11774,What parts of a human possess the highest conc...,hand and ears,hands and feet,face and hair,hand and ears,face and ears,E,hand and ears hands and feet face and hair han...,What parts of a human possess the highest conc...,0,1


In [ ]:
df[df['on'] == 0]

,prompt,A,B,C,D,E,answer,answer_all,prompt_answer_stem,labels,on
6,Organisms categorized by what species descript...,ring species,surface species,fitting species,species complex,surface species,A,ring species surface species fitting species s...,Organisms categorized by what species descript...,23,0
8,What is the stored food in a seed called?,endosperm,pollin,membrane,larval,pollin,A,endosperm pollin membrane larval pollin,What is the stored food in a seed called? endo...,19,0
9,What is controlled by both genes and experienc...,animal behaviors,reflexes,instincts,learned behaviors,reflexes,A,animal behaviors reflexes instincts learned be...,What is controlled by both genes and experienc...,23,0
16,"Fertilization is the union of a sperm and egg,...",a zygote,a cytoplasm,a nuclei,a bacteriophage,a cytoplasm,A,a zygote a cytoplasm a nuclei a bacteriophage ...,"Fertilization is the union of a sperm and egg,...",15,0
17,The plants alternation between haploid and dip...,reproduce asexually and sexually,reproduce asexually and simultaneously,reproduce sexually and autonomously,reproduce asexually and biologically,reproduce asexually and simultaneously,A,reproduce asexually and sexually reproduce ase...,The plants alternation between haploid and dip...,15,0
...,...,...,...,...,...,...,...,...,...,...,...
11751,What is used to show energy flow through the t...,biome graph,food pyramid,inverted food chain,biome graph,pyramid ecosystem,E,biome graph food pyramid inverted food chain b...,What is used to show energy flow through the t...,22,0
11756,A habitat’s features are determined mainly by ...,mixing and rainfall,oxygen and rainfall,length,mixing and rainfall,temperature and rainfall,E,mixing and rainfall oxygen and rainfall length...,A habitat’s features are determined mainly by ...,27,0
11760,"What are an important predators of mosquitoes,...",deers,primates,dogs,deers,dragonflies,E,deers primates dogs deers dragonflies,"What are an important predators of mosquitoes,...",10,0
11766,Genetic variation produced in sexual life cycl...,deconstruction,maturity,generation,deconstruction,evolution,E,deconstruction maturity generation deconstruct...,Genetic variation produced in sexual life cycl...,23,0


In [ ]:
import numpy as np
def cluster_numpy_array(data, num_clusters):
    # Perform clustering
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(data)

    # Get cluster labels
    cluster_labels = kmeans.labels_

    return cluster_labels

# Example usage
data = np.array([[2, 3], [8, 5], [3, 1], [4, 9], [7, 2]])
num_clusters = 2

cluster_labels = cluster_numpy_array(data, num_clusters)
print(cluster_labels)

[0 1 0 1 0]


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
